# 控制机械臂坐标 Control coords

In [1]:
# 导入库  import library
import os
import ipywidgets.widgets as widgets
from IPython.display import display
import time
import threading
import cv2 as cv
from pymycobot.mycobot import MyCobot
from pymycobot.genre import Angle

In [2]:
mc = MyCobot('/dev/ttyUSB0', 1000000)
g_speed = 50

Note: This class is no longer maintained since v3.6.0, please refer to the project documentation: https://github.com/elephantrobotics/pymycobot/blob/main/README.md


In [ ]:
# 重置机械臂关节角度 reset joints
button_reset = widgets.Button(     
    description='Reset',        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',
    icon='uncheck' )

# 打开机械臂电源 Power_on
button_power_on = widgets.Button(     
    description='Power_on',        
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',
    icon='uncheck' )

# 关闭机械臂电源 Power_off
button_power_off = widgets.Button(     
    description='Power_off',        
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',
    icon='uncheck' )

# 按键按下事件处理   Key press event processing
def on_button_clicked(b):
    with output:
        print("Button clicked:", b.description)
    if b.description == 'Reset':
        reset_joints()
    elif b.description == 'Power_on':
        mc.power_on()
        b.icon = 'check'
        button_power_off.icon = 'uncheck'
    elif b.description == 'Power_off':
        mc.power_off()
        b.icon = 'check'
        button_power_on.icon = 'uncheck'
    
# 关联按键事件回调 Button event callbacks
button_reset.on_click(on_button_clicked)    
button_power_on.on_click(on_button_clicked)    
button_power_off.on_click(on_button_clicked)    

In [4]:
def on_slider_gripper(angle):
    print("G7:", angle)
    mc.set_gripper_value(angle, g_speed)


def on_slider_coords(x, y, z, rx, ry, rz):
    coords = [x, y, z, rx, ry, rz]
    mc.send_coords(coords, g_speed, 0)

In [5]:
# 新建滑动条来控制舵机角度。 New sliders to control motor Angle.
slider_x = widgets.IntSlider(description='x:', value=50, min=-280, max=280, step=1, orientation='horizontal')
slider_y = widgets.IntSlider(description='y:', value=-64, min=-280, max=280, step=1, orientation='horizontal')
slider_z = widgets.IntSlider(description='z:', value=419, min=-70, max=419, step=1, orientation='horizontal')
slider_rx = widgets.IntSlider(description='rx:', value=-90 , min=-180, max=180, step=1, orientation='horizontal')
slider_ry = widgets.IntSlider(description='ry:', value=-45, min=-180, max=180, step=1, orientation='horizontal')
slider_rz = widgets.IntSlider(description='rz:', value=-90, min=-180, max=180, step=1, orientation='horizontal')
slider_g = widgets.IntSlider(description='g:', value=100 , min=30, max=100, step=1, orientation='horizontal')

widget_coords = widgets.interactive(on_slider_coords, x=slider_x, y=slider_y, z=slider_z, rx=slider_rx, ry=slider_ry, rz=slider_rz)
widget_gripper = widgets.interactive(on_slider_gripper, angle=slider_g)

box_joints = widgets.VBox([widget_gripper, widget_coords])

In [6]:
# 新建滑动条来控制运动速度。 New sliders to control speed
slider_speed = widgets.IntSlider(description='Speed:', value=50, min=1, max=100, step=1, orientation='horizontal')

def on_slider_speed(value):
    global mc, g_speed
    g_speed = value
    print("speed:", value)

widget_speed = widgets.interactive(on_slider_speed, value=slider_speed)

In [10]:
def reset_joints():
    if button_power_off.icon == 'check':
        mc.power_on()
        time.sleep(1)
    mc.send_angles([0, 0, 0, 0, 0, -45], 50)
    mc.set_gripper_value(100, 50)
    slider_x.value = 50
    slider_y.value = -64
    slider_z.value = 419
    slider_rx.value = -90
    slider_ry.value = -45
    slider_rz.value = -90
    slider_g.value = 100
    slider_speed.value = 50
    button_power_on.icon = 'check'
    button_power_off.icon = 'uncheck'
    
reset_joints()

In [11]:
button_close = widgets.Button(description='Close Camera', button_style='danger')
def button_close_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
button_close.on_click(button_close_Callback)

In [12]:
imgbox = widgets.Image(format='jpg', width=640, height=480, layout=widgets.Layout(align_self='center'))
model = 'Start'

In [13]:
# 打开摄像头 Open camera
def camera():
    global model
    capture = cv.VideoCapture(0)
    capture.set(cv.CAP_PROP_FRAME_WIDTH, 640)
    capture.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    while capture.isOpened():
        try:
            _, img = capture.read()
            if model == 'Exit':
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except:
            break
    print("capture release")
    capture.release()

In [ ]:
threading.Thread(target=camera, ).start()
output = widgets.Output()
box_group1 = widgets.VBox([widget_speed, button_reset, button_power_on, button_power_off, button_close, output])
box_display = widgets.HBox([box_group1, box_joints, imgbox])
display(box_display)

capture release
